In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from tqdm import trange
import time
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.DataFrame(columns=['INDEX','LINK','LINK_RTF'])

In [4]:
root_url = 'https://www.corteconstitucional.gov.co'

In [6]:
for i in tqdm(range(204,255)):
    
    try:
        page_url = '/relatoria/providencia.aspx?pg='+ str(i) +'&buscar=&anio=&total=35718'
        main_url = root_url + page_url

        main_html_text = requests.get(main_url).text
        main_soup = BeautifulSoup(main_html_text, 'lxml')

        docs = main_soup.find_all('div', class_ = 'grow')

        for doc in docs:
            
            
            doc_link = doc.find('a')['href']

            inner_doc_url = root_url + doc_link
            
            try:
                
                html_inner_doc = requests.get(inner_doc_url).text
                inner_doc_soup = BeautifulSoup(html_inner_doc, 'lxml')
                doc_index = doc.find('a').text
                doc_rtf_link = root_url + (inner_doc_soup.find('a')['href'])

                new_row = {'INDEX':doc_index,'LINK':inner_doc_url,'LINK_RTF':doc_rtf_link}
                data = data.append(new_row, ignore_index=True)
            
            except Exception as e:
                
                print("Error!: " + str(e))
                print('Problema en: '+ inner_doc_url)
                print('Pagina: ' + main_url)
                print('Etapa:' + str(i))
                print('*'*30)
            

    except Exception as e:
        print("Error!: " + str(e))

 57%|█████▋    | 29/51 [59:30<41:47, 113.98s/it]

Error!: HTTPSConnectionPool(host='www.corteconstitucional.gov.co', port=443): Max retries exceeded with url: /Relatoria/2005/T-298-05.htm (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4118bc4f90>: Failed to establish a new connection: [Errno 110] Connection timed out'))
Problema en: https://www.corteconstitucional.gov.co/Relatoria/2005/T-298-05.htm
Pagina: https://www.corteconstitucional.gov.co/relatoria/providencia.aspx?pg=233&buscar=&anio=&total=35718
Etapa:233
******************************


100%|██████████| 51/51 [1:45:20<00:00, 123.92s/it]


In [7]:
data

,INDEX,LINK,LINK_RTF
0,T-1056-07,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
1,T-1055-07,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
2,T-1054-07,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
3,T-1053-07,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
4,T-1052-07,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
...,...,...,...
5145,T-687-03,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5146,T-686-03,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5147,T-685-03,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5148,T-684-03,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...


In [9]:
file_name = '/content/drive/My Drive/LINKS_5.csv'
data.to_csv(file_name, encoding='utf-8', index= False)